# Proyecto 1 — Estación de llenado y taponado



Importación de las librerias

In [105]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

Definir las rutas de los archivos

In [106]:
data_path = Path("data")

telemetria_file = data_path / "telemetria.csv"
eventos_file = data_path / "eventos.csv"
botellas_file = data_path / "botellas.csv"

# FASE 1 --->Ingesta y validación (Pandas)


PASO 1: Carga y tipado de datos

    1.1 Definir tipos de datos explícitos para cada CSV utilizando un diccionario 

In [107]:

dtype_tel = {
    'temp_prod': 'float32',
    'vel_cinta': 'float32',
    'caudal': 'float32',
    'energia_kwh': 'float64'
}

dtype_evt = {
    'tipo': 'category',
    'id_botella': 'Int64'
}

dtype_pz = {
    'id_botella': 'int64',
    'peso_neto': 'float32',
    'formato': 'category'
}


    1.2 Cargar los csv en los dataFrames parseando el tiempo a datetime64

In [108]:
# Cargar telemetría con parseo de fecha
df_tel = pd.read_csv(
    telemetria_file,
    dtype=dtype_tel,
    parse_dates=['ts'],
    date_format='ISO8601'
)
# Convertir ts a UTC y establecer como índice
df_tel['ts'] = pd.to_datetime(df_tel['ts'], utc=True) # Convierte a datetime con zona horaria UTC
df_tel = df_tel.set_index('ts').sort_index() # Establece la columna de tiempo como índice del DataFrame

# Cargar eventos
df_evt = pd.read_csv(
    eventos_file,
    dtype=dtype_evt,
    parse_dates=['ts_ini', 'ts_fin'],
    date_format='ISO8601'
) # Lee el CSV y convierte automáticamente las columnas de fecha
df_evt['ts_ini'] = pd.to_datetime(df_evt['ts_ini'], utc=True) #Convierte a datetime con zona horaria UTC
df_evt['ts_fin'] = pd.to_datetime(df_evt['ts_fin'], utc=True) #Convierte a datetime con zona horaria UTC
df_evt = df_evt.sort_values('ts_ini').reset_index(drop=True)

# Cargar botellas
df_pz = pd.read_csv(
    botellas_file,
    dtype=dtype_pz,
    parse_dates=['ts_ciclo'],
    date_format='ISO8601'
)
df_pz['ts_ciclo'] = pd.to_datetime(df_pz['ts_ciclo'], utc=True) #Convierte a datetime con zona horaria UTC
df_pz = df_pz.sort_values('ts_ciclo').reset_index(drop=True) #


PASO 2. Orden y duplicados

    2.1 Telemetría

In [109]:
print("Telemetria:")
# Eliminar duplicados exactos en telemetría
duplicados_antes_tel = df_tel.duplicated().sum()
df_tel = df_tel[~df_tel.duplicated(keep='first')]

#df_evt = df_evt[~df_evt.duplicated(keep='first')]

# Verificar monotonía del índice
es_monotono_tel = df_tel.index.is_monotonic_increasing #Comprobacion de la monotonia: Los indices temporales avanzan correctamente de menor a mayor

print(f"✅ Duplicados eliminados: {duplicados_antes_tel}")
print(f"✅ Índice monótono: {es_monotono_tel}")

# Verificar si hay duplicados en el índice temporal
duplicados_index = df_tel.index.duplicated().sum()
if duplicados_index > 0:
    print(f"⚠️ Hay {duplicados_index} timestamps duplicados en el índice")
    df_tel = df_tel[~df_tel.index.duplicated(keep='first')]
else:
    print("✅ No hay timestamps duplicados")


Telemetria:
✅ Duplicados eliminados: 0
✅ Índice monótono: True
✅ No hay timestamps duplicados


    2.2 Eventos

In [110]:
print("Eventos:")
# Eliminar duplicados exactos en telemetría
dup_evt = df_evt.duplicated().sum()
df_evt = df_evt[~df_evt.duplicated(keep='first')]

# Ordenar por tiempo de inicio (y fin como desempate)
df_evt = df_evt.sort_values(['ts_ini', 'ts_fin']).reset_index(drop=True)

# Chequeos
es_monotono_evt = df_evt['ts_ini'].is_monotonic_increasing
neg_dur = (df_evt['ts_fin'] < df_evt['ts_ini']).sum()
dup_ts_ini = df_evt['ts_ini'].duplicated().sum()

print(f"Duplicados eliminados: {dup_evt}")
print(f"Orden por ts_ini monótono: {es_monotono_evt}")
print(f"Eventos con ts_fin < ts_ini: {neg_dur}")
print(f"Timestamps ts_ini duplicados: {dup_ts_ini}")


Eventos:
Duplicados eliminados: 0
Orden por ts_ini monótono: True
Eventos con ts_fin < ts_ini: 0
Timestamps ts_ini duplicados: 0


    2.3 Botellas

In [111]:
print("Botellas:")
dup_pz = df_pz.duplicated().sum()
df_pz = df_pz[~df_pz.duplicated(keep='first')]

# Ordenar por ts_ciclo
df_pz = df_pz.sort_values('ts_ciclo').reset_index(drop=True)

# Chequeos
es_monotono_pz = df_pz['ts_ciclo'].is_monotonic_increasing
dup_ts_ciclo = df_pz['ts_ciclo'].duplicated().sum()
dup_id_botella = df_pz['id_botella'].duplicated().sum()

print(f"Duplicados eliminados: {dup_pz}")
print(f"Orden por ts_ciclo monótono: {es_monotono_pz}")
print(f"Timestamps ts_ciclo duplicados: {dup_ts_ciclo}")
print(f"id_botella duplicados: {dup_id_botella}")

Botellas:
Duplicados eliminados: 0
Orden por ts_ciclo monótono: True
Timestamps ts_ciclo duplicados: 0
id_botella duplicados: 0


PASO 3: Validaciones de rango

    3.1 Marcar valores fuera de rango (sin eliminar)

In [112]:
RANGO_TEMP = (18.0,35.0)
RANGO_VEL = (0.0,0.5)
RANGO_CAUDAL = (0.0,12.0)

df_tel['fuera_RANGO_TEMP'] = (df_tel['temp_prod'] < RANGO_TEMP[0]) | (df_tel['temp_prod'] > RANGO_TEMP[1])
df_tel['fuera_RANGO_VEL'] = (df_tel['vel_cinta'] < RANGO_VEL[0]) | (df_tel['vel_cinta'] > RANGO_VEL[1])
df_tel['fuera_RANGO_CAUDAL'] = (df_tel['caudal'] < RANGO_CAUDAL[0]) | (df_tel['caudal'] > RANGO_CAUDAL[1])

n_temp_fuera = df_tel['fuera_RANGO_TEMP'].sum()
n_vel_fuera = df_tel['fuera_RANGO_VEL'].sum()
n_caudal_fuera = df_tel['fuera_RANGO_CAUDAL'].sum()

print("="*60)
print("📏 VALIDACIÓN DE RANGOS")
print("="*60)
print(f"temp_prod fuera de [{RANGO_TEMP[0]}, {RANGO_TEMP[1]}] °C: {n_temp_fuera} ({n_temp_fuera/len(df_tel)*100:.2f}%)")
print(f"vel_cinta fuera de [{RANGO_VEL[0]}, {RANGO_VEL[1]}] m/s: {n_vel_fuera} ({n_vel_fuera/len(df_tel)*100:.2f}%)")
print(f"caudal fuera de [{RANGO_CAUDAL[0]}, {RANGO_CAUDAL[1]}] ml/s: {n_caudal_fuera} ({n_caudal_fuera/len(df_tel)*100:.2f}%)")

# Mostrar estadísticas descriptivas
print("\n📊 Estadísticas descriptivas:")
print(df_tel[['temp_prod', 'vel_cinta', 'caudal', 'energia_kwh']].describe())

📏 VALIDACIÓN DE RANGOS
temp_prod fuera de [18.0, 35.0] °C: 0 (0.00%)
vel_cinta fuera de [0.0, 0.5] m/s: 0 (0.00%)
caudal fuera de [0.0, 12.0] ml/s: 0 (0.00%)

📊 Estadísticas descriptivas:
           temp_prod      vel_cinta         caudal    energia_kwh
count  129601.000000  129601.000000  129601.000000  129601.000000
mean       25.646635       0.269995       7.735915      64.876601
std         2.545803       0.092667       2.532649      37.845824
min        18.000000       0.000000       0.000000       0.000000
25%        23.802999       0.246000       7.443000      31.746546
50%        25.648001       0.284000       8.321000      64.762093
75%        27.365999       0.330000       9.101000      97.557688
max        33.855000       0.380000      11.723000     130.067508


    3.2 Validar que energia_kwh no decrece (salvo cuantización)

In [114]:
# Calcular diferencias entre valores consecutivos de energía
df_tel['delta_energia'] = df_tel['energia_kwh'].diff()

# Contar cuántas veces la energía DECRECE (delta < 0)
# Nota: diff() genera NaN en la primera fila, lo ignoramos
decrementos = (df_tel['delta_energia'] < 0).sum() 
total_cambios = df_tel['delta_energia'].notna().sum() #Va a contar cuantas veces decrece

print("="*60)
print("VALIDACIÓN DE ENERGÍA NO DECRECIENTE")
print("="*60)
print(f"Total de cambios: {total_cambios:,}")
print(f"Decrementos detectados: {decrementos} ({decrementos/total_cambios*100:.3f}%)")

# Mostrar algunos ejemplos de decrementos (si existen)
if decrementos > 0:
    print("\n Ejemplos de energía que decrece:")
    ejemplos_decremento = df_tel[df_tel['delta_energia'] < 0][['energia_kwh', 'delta_energia']].head(10)
    print(ejemplos_decremento)
    
    # Estadísticas de los decrementos
    print("\n Estadísticas de los decrementos:")
    print(df_tel[df_tel['delta_energia'] < 0]['delta_energia'].describe())
else:
    print("\n No se detectaron decrementos en energia_kwh")

VALIDACIÓN DE ENERGÍA NO DECRECIENTE
Total de cambios: 129,600
Decrementos detectados: 0 (0.000%)

 No se detectaron decrementos en energia_kwh


PASO 4: Monotonicidad de energía (corrección de decrementos)

In [116]:
# PASO 4: Corregir decrementos de energía (si los hubiera)
# Guardamos la columna original para comparación
df_tel['energia_kwh_original'] = df_tel['energia_kwh'].copy()

# Identificar decrementos
decrementos_mask = df_tel['delta_energia'] < 0
n_correcciones = decrementos_mask.sum()

if n_correcciones > 0:
    print(f"Se encontraron {n_correcciones} decrementos. Corrigiendo...")
    
    # Hacer clip de deltas negativos a 0
    df_tel['delta_energia_corregida'] = df_tel['delta_energia'].clip(lower=0)
    
    # Reconstruir energía acumulada desde el primer valor
    energia_inicial = df_tel['energia_kwh'].iloc[0]
    df_tel['energia_kwh'] = energia_inicial + df_tel['delta_energia_corregida'].fillna(0).cumsum()
    
    # Recalcular delta_energia con valores corregidos
    df_tel['delta_energia'] = df_tel['energia_kwh'].diff()
    
    print(f"{n_correcciones} correcciones aplicadas")
else:
    print("No se requieren correcciones en energia_kwh")
    print("La señal es naturalmente monótona creciente")

# Verificación final
decrementos_final = (df_tel['delta_energia'] < 0).sum()
print(f"\nVerificación final: {decrementos_final} decrementos después de corrección")

No se requieren correcciones en energia_kwh
La señal es naturalmente monótona creciente

Verificación final: 0 decrementos después de corrección
